#### retreving all dataset.tt files and their commands 

In [1]:
import glob
import re
import pandas as pd
import numpy as np

path = 'c:/users/akmaral/documents/issai/thingpedia-common-devices-master/'
file_paths = [f for f in glob.glob(path + '**/dataset.tt', recursive=True)]

In [6]:
strs = []
for file_path in file_paths:
    f = open(file_path, "r")
    file_content = f.readlines()
    strs.append(file_content)

In [7]:
strs[0]

['dataset @ca.randomfox language "en" {\n',
 '    query  := @ca.randomfox.floof()\n',
 '    #_[utterances=["fox pictures",\n',
 '                   "a picture of a fox",\n',
 '                   "a random fox",\n',
 '                   "a random fox picture"]]\n',
 '    #[id=53477582]\n',
 '    #[name="RandomFoxPicture"];\n',
 '\n',
 '}']

In [4]:
'''
for l in strs:
    for s in l:
        if 'Enum' in s:
            print("blah: ",s)
            s_temp = s.replace('Enum', 'BLAH')
            if 'Enum' in s_temp:
                print(s)
'''

'\nfor l in strs:\n    for s in l:\n        if \'Enum\' in s:\n            print("blah: ",s)\n            s_temp = s.replace(\'Enum\', \'BLAH\')\n            if \'Enum\' in s_temp:\n                print(s)\n'

In [8]:
def get_enum_values(contents, start_idx):
    l = contents[start_idx:].find("Enum(")
    r = contents[start_idx:].find(")")
    enum_values = []
    if (int(l) != -1 and int(r) != -1 and r > l): 
        enum_values = contents[start_idx:][l+len("Enum("):r]
        enum_values = enum_values.split(",")
    return enum_values

def find_var(command):
    l = command.find('$')
    r = command.find(' ', l, len(command))
    if(r == -1): r = command.find('"', l, len(command))
    if(r == -1): r = command.find(',', l, len(command))
    if(r == -1): r = command.find('.', l, len(command))
    res = command[l:r]
    return res

def subst_enum(utterances, enum_vals):
    result = []
    for command in utterances:
        if ('$' in command and len(enum_vals) > 0):
            var_name = find_var(command)
            for enum_val in enum_vals:
                command_rep = command.replace(var_name,enum_val)
                if ('$' not in command_rep): 
                    result.append(command_rep.strip().replace('"',''))
        elif ('$' not in command):
            result.append(command.strip().replace('"',''))
    return result

def get_filename(file_path):
    rev = ''
    i = int(len(file_path)-12)
    while (file_path[i] != '/'):
        rev += file_path[i]
        i -= 1
    res = ''.join(reversed(rev))
    return res

def check_str(char):
    if (char != ' ' and char != '"' and char != '(' and char != ')' and char != ',' and char != '{' and char != '}'):
        return True
    else:
        return False

def conv_num(temp):
    if temp == "eighteen degrees": return 18
    if temp == "twenty degrees": return 20
    if temp == "twenty three degrees": return 23
    if temp == "twenty five degrees": return 25
    return -1
    
def check_logic(temp1, temp2, low, high):
    t_low = conv_num(low)
    t_high = conv_num(high)
    if ("p_low" in temp1 and "p_high" in temp2 and t_low >= t_high): return False
    return True
    
def check_df(var_name, var_type, file_name):
    global var_df, df_ind
    if df_ind == 0: return True
    for ind in range(df_ind):
        if (var_df["var_name"].iloc[ind] == var_name and var_df["var_type"].iloc[ind] == var_type and var_df["file_name"].iloc[ind] == file_name):
            return False
    return True

def add_df(contents, i, j, st_ind, file_name):
    if (st_ind == j): return
    #l = contents[i:][st_ind:].find("p_")
    l = contents[i:].find("p_", st_ind, j)
    if (contents[i:][l-1] != ' ' and contents[i:][l-1] != '('): return
    res_name = ''
    res_type = ''
    while (l < j and check_str(contents[i:][l])):
        res_name += contents[i:][l]
        l += 1
    l += 2
    while (l < j and check_str(contents[i:][l])):
        res_type += contents[i:][l]
        l += 1
    global df_ind, var_df
    if ("p_" in res_name and len(res_type) > 3 and check_df(res_name, res_type, file_name)):
        var_df.loc[df_ind, "var_name"] = res_name
        var_df.loc[df_ind, "var_type"] = res_type
        var_df.loc[df_ind, "file_name"] = file_name
        var_df.loc[df_ind, "var_ind"] = df_ind+1
        df_ind += 1
        add_df(contents, i, j, l, file_name)
        
def subst_var(utterances, file_name, enum_vals):
    result = []
    for command in utterances:
        if ('$' in command):
            var_name = find_var(command)
            temp_var = var_df[var_df.file_name == file_name].copy()
            for i in range(len(temp_var)):
                df_name = temp_var["var_name"].iloc[i]
                df_values = temp_var["var_values"].iloc[i]
                l = var_name.find(df_name)
                if (l != -1 and df_values != "NA"):
                    for j in df_values:
                        command_rep = command.replace(var_name, j)
                        if ('$' not in command_rep):
                            result.append(command_rep.strip().replace('"',''))
                        else:
                            var_name_k = find_var(command_rep)
                            for k in range(len(temp_var)):                                    
                                df_name_k = temp_var["var_name"].iloc[k]
                                df_values_k = temp_var["var_values"].iloc[k]
                                t = var_name_k.find(df_name_k)
                                if (t != -1 and df_values_k != "NA"):
                                    for m in df_values_k:
                                        if(check_logic(var_name, var_name_k, j, m)):
                                            command_rep_k = command_rep.replace(var_name_k, m)
                                            if('$' not in command_rep_k):
                                                result.append(command_rep_k.strip().replace('"',''))
                                elif (t != -1 and len(enum_vals) > 0):
                                    for enum_val in enum_vals:
                                        command_rep = command.replace(var_name,enum_val)
                                        if ('$' not in command_rep): 
                                            result.append(command_rep.strip().replace('"',''))
                elif (l != -1 and len(enum_vals) > 0):
                    for enum_val in enum_vals:
                        command_rep = command.replace(var_name,enum_val)
                        if ('$' not in command_rep): 
                            result.append(command_rep.strip().replace('"',''))
                        else:
                            var_name_k = find_var(command_rep)
                            for k in range(len(temp_var)):
                                df_name_k = temp_var["var_name"].iloc[k]
                                df_values_k = temp_var["var_values"].iloc[k]
                                t = var_name_k.find(df_name_k)
                                if (t != -1 and df_values_k != "NA"):
                                    for m in df_values_k:
                                        if(check_logic(var_name, var_name_k, j, m)):
                                            command_rep_k = command_rep.replace(var_name_k, m)
                                            if('$' not in command_rep_k):
                                                result.append(command_rep_k.strip().replace('"',''))
                            #temp_res = subst_var(command_rep, file_name)
                            #result = result + temp_res
        #elif ('$' not in command):
        #    result.append(command.strip().replace('"',''))
    return result

def collect_variables(file_paths, command_type):
    for file_path in file_paths:
        f = open(file_path, "r")
        file_content = f.read()
        command_start_idx_lst = [i for i in range(len(file_content)) if file_content.startswith(command_type, i)]
        file_name = get_filename(file_path)
        for i in command_start_idx_lst:   
            j = file_content[i:].find("#_[utterances=[")
            add_df(file_content, i, j, 0, file_name)
                
def collect_commands(file_paths, command_type):
    commands_list = []
    for file_path in file_paths:
        f = open(file_path, "r")
        file_content = f.read()
        command_start_idx_lst = [i for i in range(len(file_content)) if file_content.startswith(command_type, i)]
        variable_set = []
        file_name = get_filename(file_path)
        for i in command_start_idx_lst:   
            j = file_content[i:].find("#_[utterances=[")
            k = file_content[i:].find("]]")
            utterances = file_content[i:][j+len("#_[utterances=["):k]
            utterances = utterances.split(',')
            commands = []
            enum_vals = get_enum_values(file_content, i)
            commands = subst_var(utterances, file_name, enum_vals)
            '''
            if(len(enum_vals) > 0): 
                commands = subst_enum(utterances, enum_vals)
            else: 
                #commands = [action_str.strip().replace('"','') for action_str in utterances if (action_str.strip().replace('"','') and ('$' not in action_str))]
                commands = subst_var(utterances, file_name)
            '''
            commands_list = commands_list + commands
    return commands_list   

In [9]:
df_ind = 0;
var_df = pd.DataFrame(columns = ["var_name","var_type","file_name", "var_ind", "var_values"])

collect_variables(file_paths, 'action')
collect_variables(file_paths, 'query')

for i in range(len(var_df)):
    var_df.loc[i, "var_values"] = "NA"
'''
var_df.loc[39, "var_values"] = ["zero", "fifty", "eighty", "hundred"]
var_df.loc[40, "var_values"] = ["eighteen degrees", "twenty degrees", "twenty three degrees", "twenty five degrees"]
var_df.loc[41, "var_values"] = ["eighteen degrees", "twenty degrees", "twenty three degrees", "twenty five degrees"]
var_df.loc[42, "var_values"] = ["eighteen degrees", "twenty degrees", "twenty three degrees", "twenty five degrees"]
var_df.loc[47, "var_values"] = ["zero", "fifty", "eighty", "hundred"]
var_df.loc[72, "var_values"] = ["eighteen degrees", "twenty degrees", "twenty three degrees", "twenty five degrees"]
var_df.loc[84, "var_values"] = ["one", "two", "five"]
var_df.loc[85, "var_values"] = ["pi", "three", "seven"]
var_df.loc[86, "var_values"] = ["twenty nineteen", "twenty twenty", "twenty twelve"]
var_df.loc[88, "var_values"] = ["one gigabyte", "five gigabytes", "hundred megabytes"]
var_df.loc[97, "var_values"] = ["five", "ten", "three"]
var_df.loc[123, "var_values"] = ["three", "five", "ten"]
var_df.loc[135, "var_values"] = ["three", "five", "ten"]
var_df.loc[136, "var_values"] = ["hundred", "five hundred", "thousand"]
var_df.loc[143, "var_values"] = ["To do list", "Work list", "Homework list"]
var_df.loc[144, "var_values"] = ["three days", "seven days", "ten days"]
var_df.loc[32, "var_values"] = ["Jingle Bells", "Despacito", "Rolling in the deep"]
var_df.loc[33, "var_values"] = ["thirty", "ten", "fifty"]
var_df.loc[36, "var_values"] = ["Favourite songs", "Dance mood", "Sad songs"]
var_df.loc[37, "var_values"] = ["Adele", "Frank Sinatra", "David Bowie"]
var_df.loc[38, "var_values"] = ["Abbey Road", "Parachutes", "Meteora"]
var_df.loc[146, "var_values"] = ["Family", "Friends", "Trips"]
var_df.loc[159, "var_values"] = ["Physics", "Mathematics", "Computer Science", "Economics"]
var_df.loc[87, "var_values"] = ["pdf", "jpg", "doc"]
var_df.loc[98, "var_values"] = ["Normal", "Clarendon", "Gingham"]
var_df.loc[106, "var_values"] = ["English", "Spanish", "Russian"]
var_df.loc[145, "var_values"] = ["Bitcoin", "Litecoin", "Ethereum"]
'''
var_df.loc[90, "var_values"] = ["New York", "London", "Tokyo"]
var_df.loc[154, "var_values"] = ["living room", "dining room", "bathroom"]
var_df.loc[152, "var_values"] = ["bedroom", "kitchen", "guest room"]

commands_action =  collect_commands(file_paths, 'action')
commands_query =  collect_commands(file_paths, 'query')

print("number of all action commands: ",len(commands_action))
print("number of all unique action commands: ",len(set(commands_action)))
#print("here is the first action command: ", commands_action[0])
print("number of all query commands: ",len(commands_query))
print("number of all unique query commands: ",len(set(commands_query)))
print("here is the first query command: ", commands_query[0])
var_df = var_df.set_index('var_ind')
var_df.to_csv('csvs/additional_commands_v2.csv')

number of all action commands:  124
number of all unique action commands:  112
number of all query commands:  265
number of all unique query commands:  265
here is the first query command:  a random cat fact


In [10]:
#check if there are numbers in any command
print([c for c in commands_action if (len(re.findall(r'\d+',c))>0)])
print([c for c in commands_query if (len(re.findall(r'\d+',c))>0)])

[]
[]


In [11]:
#removing short commands
commands_query_f = [l for l in commands_query if (len(l)>2)]
commands_query_f = [l for l in commands_query_f if (len(l.split())>1)]

In [12]:
new_commands_query_f = []
for c in commands_query_f:
    if "matrix67" in c:
        c = c.replace('matrix67','matrix sixty seven')
        print(c)
    if "cs294" in c:
        c = c.replace('cs294','cs two nine four ')
        print(c)
    new_commands_query_f.append(c)

In [13]:
all_commands =  list(set(commands_action))+ list(set(new_commands_query_f))
print(len(all_commands))

377


#### saving as .csv file

In [36]:
import pandas as pd
df = pd.DataFrame()
df["command_name"] = all_commands
df["command_id"] = range(1,len(all_commands)+1)
df["num_chars"] = [len(command) for command in all_commands]
df = df.set_index('command_id')
df.to_csv('csvs/additional_commands_v1.csv')
df.head(10)

,command_name,num_chars
command_id,,
1,meme template named Tokyo,25
2,meme templates in imgflip whose name contains ...,52
3,if my guest room heat sensor is hot or cold,43
4,if the room is occupied according to my living...,58
5,meme templates in imgflip whose name contains ...,51
6,whether the living room is detected as occupie...,81
7,if the guest room heat sensor is hot,36
8,if the bedroom heat sensor is hot,33
9,whether my living room sensor says the bedroom...,67


In [ ]:
! cat csvs/additional_commands_v1.csv | tr '_' ' ' > csvs/additional_commands_v3.csv

 ’®¬ ў гбва®©бвўҐ C Ё¬ҐҐв ¬ҐвЄг Windows 
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 12BE-D1B7

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\Akmaral\Documents\ISSAI\Madina\thermal_visual_audio_db-master\text_commands_gen

13.12.2019  14:31    <DIR>          .
13.12.2019  14:31    <DIR>          ..
13.12.2019  12:36    <DIR>          .ipynb_checkpoints
13.12.2019  12:36            21я476 commands_expr.ipynb
13.12.2019  12:36            10я380 commands_preprocess.ipynb
13.12.2019  12:36            13я439 commands_preprocess_v1.ipynb
13.12.2019  12:36            28я995 commands_preprocess_v1_1.ipynb
13.12.2019  12:36            24я767 commands_preprocess_v1_2.ipynb
13.12.2019  14:31            24я549 commands_preprocess_v1_3.ipynb
13.12.2019  14:29    <DIR>          csvs
13.12.2019  12:37    <DIR>          thingpedia-common-devices-master
13.12.2019  12:36             6я148 _DS_Store
               7 д ©«®ў        129я754 Ў ©в
               5 Ї Ї®Є  103я741я300я736 Ў ©в бў®Ў®¤­®


In [ ]:
! head csvs/additional_commands_v3.csv

In [ ]:
! cat csvs/additional_commands_v3.csv | grep '_'

In [ ]:
! cat csvs/additional_commands_v3.csv | sed "s/zero %/zero percent/g" | sed 's/%/percents/g' > csvs/additional_commands_v4.csv

In [ ]:
! cat csvs/additional_commands_v4.csv | grep 'percent' 

In [ ]:
! cat csvs/action_phrases_copy.txt | tr '_' ' ' > csvs/action_phrases

#### saving as .mat file

In [ ]:
import scipy.io as sio
sio.savemat('commands.mat', {"command_name": all_commands, 
                             "command_id" : range(len(all_commands)),
                             "num_chars": [len(command) for command in all_commands]}) 

In [ ]:
var_name = 'p_state'
temp_df = var_df[var_df.var_name == var_name].copy()

In [ ]:
a  =  np.array([1,2,3,4,5,6,7])
a[a>5]

In [ ]:
temp_df.var_type.iloc[0] = 'Blah'

In [ ]:
temp_df

In [ ]:
var_df[var_df.var_name == var_name]
#for i in range(len(temp_df)):
#    print(temp_df["var_type"].iloc[i])